In [1]:
# Import the necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

2024-03-01 03:20:35.751243: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-01 03:20:35.780440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-01 03:20:36.304161: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Preprocess the train and validation datasets
train_dir = 'dataset/train'
validation_dir = 'dataset/validation'

train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary')

Found 8448 images belonging to 2 classes.
Found 2699 images belonging to 2 classes.


In [3]:
# Using transfer learning create a derived model. 
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

2024-03-01 03:20:36.953200: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-01 03:20:36.971694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-01 03:20:36.971855: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# Train the dataset.
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator)

Epoch 1/10


2024-03-01 03:20:38.367290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-03-01 03:20:40.590546: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-03-01 03:20:40.884040: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:231] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.9
2024-03-01 03:20:40.884058: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:234] Used ptxas at ptxas
2024-03-01 03:20:40.884103: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:317] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.

264/264 [==============================] - ETA: 0s - loss: 0.3346 - accuracy: 0.8423

2024-03-01 03:23:15.700079: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


264/264 [==============================] - 205s 768ms/step - loss: 0.3346 - accuracy: 0.8423 - val_loss: 0.2493 - val_accuracy: 0.8948
Epoch 2/10
264/264 [==============================] - 207s 784ms/step - loss: 0.2669 - accuracy: 0.8728 - val_loss: 0.2360 - val_accuracy: 0.9000
Epoch 3/10
264/264 [==============================] - 201s 763ms/step - loss: 0.2438 - accuracy: 0.8861 - val_loss: 0.2624 - val_accuracy: 0.8681
Epoch 4/10
264/264 [==============================] - 201s 764ms/step - loss: 0.2276 - accuracy: 0.8916 - val_loss: 0.2204 - val_accuracy: 0.9040
Epoch 5/10
264/264 [==============================] - 207s 787ms/step - loss: 0.2123 - accuracy: 0.9002 - val_loss: 0.2086 - val_accuracy: 0.9107
Epoch 6/10
264/264 [==============================] - 202s 765ms/step - loss: 0.2007 - accuracy: 0.9084 - val_loss: 0.2287 - val_accuracy: 0.8859
Epoch 7/10
264/264 [==============================] - 208s 788ms/step - loss: 0.1925 - accuracy: 0.9135 - val_loss: 0.2338 - val_accura

In [5]:
# Save the model
model.save('model.h5')